In [2]:
%matplotlib notebook
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from itertools import groupby
import wotan

In [4]:
filename = "data/kplr004940201-2013131215648_llc.fits"

fits.info(filename)

Filename: data/kplr004940201-2013131215648_llc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   ()      
  1  LIGHTCURVE    1 BinTableHDU    155   1556R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        48   (6, 4)   int32   


In [121]:
with fits.open(filename) as hdulist:
    header1 = hdulist[1].header

print(repr(header1))

XTENSION= 'BINTABLE'           / marks the beginning of a new HDU               
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  100 / length of first array dimension                
NAXIS2  =                 1556 / length of second array dimension               
PCOUNT  =                    0 / group parameter count (not used)               
GCOUNT  =                    1 / group count (not used)                         
TFIELDS =                   20 / number of table fields                         
TTYPE1  = 'TIME    '           / column title: data time stamps                 
TFORM1  = 'D       '           / column format: 64-bit floating point           
TUNIT1  = 'BJD - 2454833'      / column units: barycenter corrected JD          
TDISP1  = 'D14.7   '           / column display format                          
TTYPE2  = 'TIMECORR'        

In [120]:
with fits.open(filename, mode="readonly") as hdulist:
    # Read in the "BJDREF" which is the time offset of the time array.
    bjdrefi = hdulist[1].header['BJDREFI']
    bjdreff = hdulist[1].header['BJDREFF']

    # Read in the columns of data.
    times = hdulist[1].data['time']
    sap_fluxes = hdulist[1].data['SAP_FLUX']

In [10]:
bjds = times + bjdrefi + bjdreff

plt.figure(figsize=(9,4))

# Plot the time, uncorrected and corrected fluxes.
plt.plot(bjds, sap_fluxes, '-k', label='SAP Flux')

plt.title('Kepler Light Curve')
plt.legend()
plt.xlabel('Time (days)')
plt.ylabel('Flux (electrons/second)')
plt.show()

<IPython.core.display.Javascript object>

In [75]:
flatten_lc = wotan.flatten(bjds, sap_fluxes, window_length=0.5, method='biweight')

In [81]:
plt.figure(figsize=(9,4))

plt.plot(bjds, flatten_lc, '-k', label='Detrended Flux')

plt.title('Kepler Light Curve')
plt.legend()
plt.xlabel('Time (days)')
plt.ylabel('Detrended Flux (electrons/second)')
plt.show()

<IPython.core.display.Javascript object>

In [86]:
rear_codes = {
    "EB_Q0_long"    : "-2009131105131_llc.fits",
    "EB_Q1_long"    : "-2009166043257_llc.fits",
    "EB_Q2_long"    : "-2009259160929_llc.fits",
    "EB_Q3_long"    : "-2009350155506_llc.fits",
    "EB_Q4_long"    : "-2010078095331_llc.fits",
    "EB_Q5_long"    : "-2010174085026_llc.fits",
    "EB_Q6_long"    : "-2010265121752_llc.fits",
    "EB_Q7_long"    : "-2010355172524_llc.fits",
    "EB_Q8_long"    : "-2011073133259_llc.fits",
    "EB_Q9_long"    : "-2011177032512_llc.fits",
    "EB_Q10_long"   : "-2011271113734_llc.fits",
    "EB_Q11_long"   : "-2012004120508_llc.fits",
    "EB_Q12_long"   : "-2012088054726_llc.fits",
    "EB_Q13_long"   : "-2012179063303_llc.fits",
    "EB_Q14_long"   : "-2012277125453_llc.fits",
    "EB_Q15_long"   : "-2013011073258_llc.fits",
    "EB_Q16_long"   : "-2013098041711_llc.fits",
    "EB_Q17_long"   : "-2013131215648_llc.fits"
}

In [87]:
import os
def get_matching_files(data_path, front_code):
    matches = []
    for f in rear_codes.keys():
        temp_path = f + "/" + front_code + rear_codes[f]
        if os.path.exists(data_path + "/" + temp_path):
            matches.append(temp_path)
    return matches

In [17]:
from get_threshold import get_threshold

In [16]:
file = "kplr002161623.fits"
t1 = Table.read("data/combined" + "/" + file, format="fits")

plt.figure(figsize=(9,4))
flattened_lc = wotan.flatten(t1["TIME"], t1["SAP_FLUX"], window_length=0.5, method='biweight')
median = np.nanmedian(flattened_lc)
std = np.nanstd(flattened_lc)
threshold = get_threshold(median, std)
plt.plot(t1["TIME"], flattened_lc, '-k', label='Detrended Flux')
plt.plot(plt.axis()[:2], [threshold, threshold], "-b")
plt.title('Kepler Light Curve')
plt.legend()
plt.xlabel('Time (days)')
plt.ylabel('Detrended Flux (electrons/second)')
plt.show()

<IPython.core.display.Javascript object>